# Minimal code for Figure S2e polysome profiling panels

In [ ]:
# turn off warnings
options(warn = -1)
suppressPackageStartupMessages({
  library(tidyverse)
  library(rasilabRtemplates)
})

cbPalette_12 <- c(
  "#999999", "#CC6677", "#88CCEE", "#661100", "#117733", "#332288", 
  "#999933","#AA4499", "#44AA99", "#882255", "#6699CC", "#DDCC77"
)

theme_set(theme_rasilab() +
  theme(
    axis.line = element_line(color = "grey"),
    axis.title.y = element_text(margin = margin(r = 10)),
    axis.title.x = element_text(margin = margin(t = 10))
  ))

In [ ]:
if (!file.exists('../figures')){
    dir.create('../figures')
}

## Read annotations

In [ ]:
annotations <- read_csv("../annotations/sampleannotations.csv", col_types = cols(.default = "c")) %>% 
  print()

## Read data

In [ ]:
counts <- list.files("../../../../data/polysome_profiling/polysome_relic_hits/", pattern = "\\d\\.csv", full.names = T) %>% 
  enframe("sno", "file") %>% 
  mutate(data = map(file, . %>%
                      read_csv(., skip = 45, show_col_types = F) %>%
                      select(`Position(mm)`, Absorbance, `Fraction Number`)
                      ))%>%
  unnest() %>%
  rename(AbsA = Absorbance,
       Fraction = `Fraction Number`,
       Position = `Position(mm)`) %>%
  mutate(file = str_extract(file, "20241104_i287_\\d.csv")) %>%
  left_join(annotations) %>%
  filter(!is.na(sample)) %>%
  select(-sno, -file) %>%  
  print()

## Standardize traces

In [ ]:
standardized <- counts %>%
    # remove all initial values < 0
    filter(!(AbsA < 0 & Position < 10)) %>%
    # make baseline equal to 0
    group_by(sample) %>%
    mutate(AbsA = AbsA - min(AbsA)) %>%
    ungroup()

## Calculate P/M ratio

In [ ]:
PbyM <- standardized %>%
    group_by(sample) %>%
    # Pick a minimum position just short of 40S and 60S peaks
    summarize(
      M = sum(AbsA[Position > 10 & Position < 28]),
      P = sum(AbsA[Position >= 28 & Position < 80])
    ) %>% 
    mutate(PbyM = P/M)  %>% 
    mutate(PbyM = PbyM / PbyM[sample == "sgFLUC"])  %>% 
    mutate(PbyM = round(PbyM, 2))  %>% 
    arrange(desc(PbyM))

PbyM

## Normalize to approximate value of 80S

In [ ]:
normalized_counts <- standardized %>%
    group_by(sample) %>%
    mutate(AbsA = AbsA - min(AbsA)) %>%
    mutate(MonoAbsA = max(AbsA[Position > 10])) %>%
    mutate(AbsA = AbsA/MonoAbsA)

## Plot normalized absorbance

In [ ]:
options(repr.plot.width=6, repr.plot.height=3)

# put them all on one plot
normalized_counts %>%
  filter(Position > 10 & Position < 80, ! sample %in% c("sgRPL5")) %>% 
  left_join(PbyM, by = "sample") %>%
  mutate(sample = paste0(sample, ", ", PbyM)) %>%
  ggplot(aes(x = Position, y = AbsA, color = sample)) +
  geom_line(linewidth = 0.8) +
  xlab("Position (mm)") +
  ylab("A260 (A.U.)") +
  scale_y_continuous(limits = c(0, 1.2), breaks = c(0, 0.4, 0.8, 1.2)) +
  scale_color_manual(values = cbPalette_12[c(2,1,3,4,5)])

ggsave("../figures/normalized_trace_with_PbyM.pdf", width = 6, height = 3, units = "in")